In [ ]:
import os
from pyspark.rdd import RDD
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc
from pyspark.sql import functions as F

In [ ]:
head, tail = os.path.split(os.getcwd())
data_dir = os.path.join(head, 'data')
data_raw_dir = os.path.join(data_dir, 'raw')
DATA_INTERIM_DIR = os.path.join(data_dir, 'interim')
data_raw_dir

In [ ]:
def init_spark():
    spark = SparkSession \
        .builder \
        .appName("Python Spark SQL basic example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()
    return spark

In [ ]:
spark = init_spark()

data_combined_2015 = spark.read.parquet(os.path.join(DATA_INTERIM_DIR, 'weather_2015.data'))

data_combined_2016 = spark.read.parquet(os.path.join(DATA_INTERIM_DIR, 'weather_2016.data'))
# data_combined_2016 = data_combined_2016.na.fill(0)
data_combined_2017 = spark.read.parquet(os.path.join(DATA_INTERIM_DIR, 'weather_2017.data'))
# data_combined_2017 = data_combined_2017.na.fill(0)
data_combined_2018 = spark.read.parquet(os.path.join(DATA_INTERIM_DIR, 'weather_2018.data'))
# data_combined_2018 = data_combined_2018.na.fill(0)
data_combined_2019 = spark.read.parquet(os.path.join(DATA_INTERIM_DIR, 'weather_2019.data'))
# data_combined_2019 = data_combined_2019.na.fill(0)
data_combined_2020 = spark.read.parquet(os.path.join(DATA_INTERIM_DIR, 'weather_2020.data'))
# data_combined_2020 = data_combined_2020.na.fill(0)
l = [data_combined_2015, data_combined_2016, data_combined_2017, data_combined_2018, data_combined_2019, data_combined_2020]

In [ ]:
null_cols = []
for col_name in data_combined_2015.schema.names:
    if data_combined_2015.filter(data_combined_2015[col_name].isNull()).count() > 0:
        print(col_name)
        null_cols.append(col_name)

In [ ]:
for null_col_name in null_cols:
    data_combined_2015 = data_combined_2015.fillna({null_col_name:'0'})

In [ ]:
for each_file in l:
    null_cols = []
    for col_name in each_file.schema.names:
        if each_file.filter(each_file[col_name].isNull()).count() > 0:
            null_cols.append(col_name)
    
    for null_col_name in null_cols:
        each_file = each_file.fillna({null_col_name:'0'})

In [ ]:
df_final = l[0].unionByName(l[1])

In [ ]:
for i in range(2, len(l)):
    df_final = df_final.unionByName(l[i])

In [ ]:
print(data_combined_2015.count())
print(data_combined_2016.count())
print(data_combined_2017.count())
print(data_combined_2018.count())
print(data_combined_2019.count())
print(data_combined_2020.count())
print(df_final.count())

In [ ]:
df_final.head(5)